# Current Target Table Contents
Execute ELT (Azure Data Factory pipeline, etc.)
Run this after each simulated load

### `PersonRigid`
Note truncated data, loss of unicode characters, etc.

In [111]:
select * from dbo.PersonRigid order by LastName, FirstName

(0 rows affected)

Total execution time: 00:00:00.024

Id,FirstName,MiddleInitial,LastName


### `PersonFlexible`
Note fidelity as columns were altered to fit incoming data

In [112]:
select * from dbo.PersonFlexible order by LastName, FirstName

(0 rows affected)

Total execution time: 00:00:00.022

Id,FirstName,MiddleInitial,LastName,MiddleName


Compare schemas of both tables

In [113]:
with cte as (
    select TableName, ColumnName, DefinitionSql from dbo.uftGetTableDefinition('dbo', 'PersonRigid', 0)
    union
    select TableName, ColumnName, DefinitionSql from dbo.uftGetTableDefinition('dbo', 'PersonFlexible', 0)
)

select      TableName, DefinitionSql
        ,   case when lag(ColumnName) over (order by ColumnName, TableName) is null then ''
                 when lag(ColumnName) over (order by ColumnName, TableName) <> ColumnName then ''
                 when lag(ColumnName) over (order by ColumnName, TableName) = ColumnName and lag(DefinitionSql) over (order by ColumnName, TableName) = DefinitionSql 
                 then '' else '***' end match
from        cte
order by    DefinitionSql, TableName

(9 rows affected)

Total execution time: 00:00:00.028

TableName,DefinitionSql,match
PersonFlexible,[FirstName] nvarchar(20) NULL,
PersonRigid,[FirstName] varchar(10) NULL,***
PersonFlexible,[Id] int NOT NULL,
PersonRigid,[Id] int NOT NULL,
PersonFlexible,[LastName] nvarchar(71) NULL,
PersonRigid,[LastName] varchar(15) NULL,***
PersonFlexible,[MiddleInitial] char(1) NULL,
PersonRigid,[MiddleInitial] char(1) NULL,
PersonFlexible,[MiddleName] nvarchar(9) NULL,


## Check audit history for any changes made

In [114]:
select      *, current_timestamp TimeNow
from        audit.SchemaDrift for system_time all
where       SchemaName = 'dbo'
    and     TableName in ('PersonRigid', 'PersonFlexible')
order by    SchemaName, TableName, ColumnName, SysStartTime

(12 rows affected)

Total execution time: 00:00:00.040

SchemaName,TableName,ColumnName,Definition,PriorDefinition,ActionReason,DdlProposed,SysStartTime,SysEndTime,TimeNow
dbo,PersonFlexible,[FirstName],[FirstName] varchar(20) NULL,[FirstName] varchar(10) NULL,DDL.SMALLER_STRING,NULL,2023-10-26 20:14:23.0624119,2023-10-26 20:14:23.0936842,2023-10-26 20:14:35.430
dbo,PersonFlexible,[FirstName],[FirstName] nvarchar(20) NULL,[FirstName] varchar(20) NULL,DDL.UNICODE_TO_ASCII,NULL,2023-10-26 20:14:23.1093068,9999-12-31 23:59:59.9999999,2023-10-26 20:14:35.430
dbo,PersonFlexible,[LastName],[LastName] varchar(25) NULL,[LastName] varchar(15) NULL,DDL.SMALLER_STRING,NULL,2023-10-26 20:14:23.0624119,2023-10-26 20:14:23.0936842,2023-10-26 20:14:35.430
dbo,PersonFlexible,[LastName],[LastName] varchar(71) NULL,[LastName] varchar(25) NULL,DDL.SMALLER_STRING,NULL,2023-10-26 20:14:23.0936842,2023-10-26 20:14:23.1249137,2023-10-26 20:14:35.430
dbo,PersonFlexible,[LastName],[LastName] nvarchar(71) NULL,[LastName] varchar(71) NULL,DDL.UNICODE_TO_ASCII,NULL,2023-10-26 20:14:23.1249137,9999-12-31 23:59:59.9999999,2023-10-26 20:14:35.430
dbo,PersonFlexible,[MiddleName],[MiddleName] varchar(9) NULL,NULL,DDL.NO_DEST,NULL,2023-10-26 20:14:23.0780570,2023-10-26 20:14:23.1093068,2023-10-26 20:14:35.430
dbo,PersonFlexible,[MiddleName],[MiddleName] nvarchar(9) NULL,[MiddleName] varchar(9) NULL,DDL.UNICODE_TO_ASCII,NULL,2023-10-26 20:14:23.1249137,9999-12-31 23:59:59.9999999,2023-10-26 20:14:35.430
dbo,PersonRigid,[FirstName],[FirstName] varchar(10) NULL,NULL,DDL.SMALLER_STRING,alter table [dbo].[PersonRigid] alter column [FirstName] varchar(20) NULL,2023-10-26 20:14:20.8124127,2023-10-26 20:14:20.8436625,2023-10-26 20:14:35.430
dbo,PersonRigid,[FirstName],[FirstName] varchar(10) NULL,NULL,DDL.UNICODE_TO_ASCII,alter table [dbo].[PersonRigid] alter column [FirstName] nvarchar(10) NULL,2023-10-26 20:14:20.8436625,9999-12-31 23:59:59.9999999,2023-10-26 20:14:35.430
dbo,PersonRigid,[LastName],[LastName] varchar(15) NULL,NULL,DDL.SMALLER_STRING,alter table [dbo].[PersonRigid] alter column [LastName] varchar(25) NULL,2023-10-26 20:14:20.8124127,2023-10-26 20:14:20.8436625,2023-10-26 20:14:35.430
